In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pyyaml==5.1 pycocotools>=2.0.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version

1.6.0+cu101 True
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
assert torch.__version__.startswith("1.6")
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html


In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
%cd drive/My Drive/cyc3600

/content/drive/My Drive/cyc3600


In [ ]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances('cyclone_ds_train', {}, 'ftrain80.json', 'train')

In [ ]:
cyclone_metadata = MetadataCatalog.get('cyclone_ds_train')

In [ ]:
dataset_dicts = DatasetCatalog.get('cyclone_ds_train')

[09/15 23:57:53 d2.data.datasets.coco]: Loaded 3977 images in COCO format from ftrain80.json


In [ ]:
import random
from detectron2.utils.visualizer import Visualizer

for d in random.sample(dataset_dicts, 10):
    print(d['file_name'])
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=cyclone_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import os

cfg = get_cfg()
#cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/retinanet_R_101_FPN_3x.yaml"))
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_DC5_3x.yaml"))
# cfg.merge_from_file(
#     "./detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
# )
#cfg.DATASETS.TRAIN = ("cyclone_ds_train",)
#cfg.DATASETS.TEST = ()  # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 2
#cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"  # initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 1
cfg.SOLVER.BASE_LR = 0.002
cfg.SOLVER.MAX_ITER = (
    2000
)  # 300 iterations seems good enough, but you can certainly train longer
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = (
    1280
)  # faster, and good enough for this toy dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4  # 3 classes (data, fig, hazelnut)
cfg.INPUT.MAX_SIZE_TEST = 3600
cfg.INPUT.MAX_SIZE_TRAIN = 3600
cfg.INPUT.MIN_SIZE_TEST = 1201
cfg.INPUT.MIN_SIZE_TRAIN = 1201
cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[60,70,80,90]]

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import os

cfg = get_cfg()
#cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/retinanet_R_101_FPN_3x.yaml"))
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_DC5_3x.yaml"))
# cfg.merge_from_file(
#     "./detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
# )
cfg.DATASETS.TRAIN = ("cyclone_ds_train",)
cfg.DATASETS.TEST = ()  # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 2
#cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"  # initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 1
cfg.SOLVER.BASE_LR = 0.002
cfg.SOLVER.MAX_ITER = (
    1800
)  # 300 iterations seems good enough, but you can certainly train longer
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = (
    1280
)  # faster, and good enough for this toy dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4  # 3 classes (data, fig, hazelnut)
cfg.INPUT.MAX_SIZE_TEST = 3600
cfg.INPUT.MAX_SIZE_TRAIN = 3600
cfg.INPUT.MIN_SIZE_TEST = 1201
cfg.INPUT.MIN_SIZE_TRAIN = 1201
cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[60,70,80,90]]


os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[09/14 12:32:13 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): ResNet(
    (stem): BasicStem(
      (conv1): Conv2d(
        3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
        (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
      )
    )
    (res2): Sequential(
      (0): BottleneckBlock(
        (shortcut): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
        (conv1): Conv2d(
          64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv2): Conv2d(
          64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv3): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
 

/usr/local/lib/python3.6/dist-packages/detectron2/layers/wrappers.py:226: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  return x.nonzero().unbind(1)


[09/14 12:33:11 d2.utils.events]:  eta: 1:19:37  iter: 19  total_loss: 1.976  loss_cls: 1.111  loss_box_reg: 0.000  loss_rpn_cls: 0.697  loss_rpn_loc: 0.214  time: 2.7005  data_time: 0.0448  lr: 0.000040  max_mem: 7812M
[09/14 12:34:02 d2.utils.events]:  eta: 1:16:48  iter: 39  total_loss: 0.725  loss_cls: 0.089  loss_box_reg: 0.006  loss_rpn_cls: 0.343  loss_rpn_loc: 0.227  time: 2.6143  data_time: 0.0030  lr: 0.000080  max_mem: 7812M
[09/14 12:34:52 d2.utils.events]:  eta: 1:13:38  iter: 59  total_loss: 0.438  loss_cls: 0.091  loss_box_reg: 0.013  loss_rpn_cls: 0.161  loss_rpn_loc: 0.180  time: 2.5801  data_time: 0.0033  lr: 0.000120  max_mem: 7812M
[09/14 12:35:42 d2.utils.events]:  eta: 1:12:01  iter: 79  total_loss: 0.341  loss_cls: 0.074  loss_box_reg: 0.025  loss_rpn_cls: 0.150  loss_rpn_loc: 0.096  time: 2.5553  data_time: 0.0031  lr: 0.000160  max_mem: 7812M
[09/14 12:36:31 d2.utils.events]:  eta: 1:10:42  iter: 99  total_loss: 0.381  loss_cls: 0.105  loss_box_reg: 0.050  loss

In [ ]:
register_coco_instances('cyclone_ds_val', {}, 'fval80.json', 'val')

In [ ]:
val_metadata = MetadataCatalog.get('cyclone_ds_val')
val_dicts = DatasetCatalog.get('cyclone_ds_val')

[09/14 13:46:58 d2.data.datasets.coco]: Loaded 1439 images in COCO format from fval80.json


In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
#cfg.MODEL.WEIGHTS = os.path.join('b80-R101-DC5-3x', "model_final.pth")
#cfg.MODEL.WEIGHTS = '/content/drive/My Drive/cyc1440/output/model_final.pth'
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.4 # set the testing threshold for this model
cfg.DATASETS.TEST = ("cyclone_ds_val", )
predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.utils.visualizer import ColorMode

for d in random.sample(val_dicts, 1000):
    print(d['file_name'])
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    print(outputs)

val/ssd_20190704_06.png
{'instances': Instances(num_instances=1, image_height=1201, image_width=3600, fields=[pred_boxes: Boxes(tensor([[2269.2249,  410.8936, 2355.1941,  487.7227]], device='cuda:0')), scores: tensor([0.5607], device='cuda:0'), pred_classes: tensor([2], device='cuda:0')])}
val/ssd_20191013_12.png
{'instances': Instances(num_instances=2, image_height=1201, image_width=3600, fields=[pred_boxes: Boxes(tensor([[2426.4971,  344.9706, 2507.3135,  426.8687],
        [3417.5112,  442.3873, 3499.0869,  523.3585]], device='cuda:0')), scores: tensor([0.6788, 0.5701], device='cuda:0'), pred_classes: tensor([1, 1], device='cuda:0')])}
val/ssd_20190615_18.png
{'instances': Instances(num_instances=1, image_height=1201, image_width=3600, fields=[pred_boxes: Boxes(tensor([[614.2153, 354.9939, 697.0400, 430.6292]], device='cuda:0')), scores: tensor([0.6490], device='cuda:0'), pred_classes: tensor([1], device='cuda:0')])}
val/ssd_20190411_06.png
{'instances': Instances(num_instances=0, i

In [ ]:
# from detectron2.modeling import build_model
# model = build_model(cfg)

# from detectron2.checkpoint import DetectionCheckpointer
# DetectionCheckpointer(model).load('b80-R101-DC5-3x/model_final.pth')

temp_val_dict = val_dicts[:2]

print(temp_val_dict)
for each in temp_val_dict:
    print(each)

# model.eval()
# with torch.no_grad():
#     outputs = model(val_dicts[:10])

# for d in random.sample(val_dicts, 10):
#     print(d)
#     im = cv2.imread(d["file_name"])
#     outputs = predictor(im)
#     print(outputs)


[{'file_name': 'val/ssd_20190101_00.png', 'height': 1201, 'width': 3600, 'image_id': 0, 'annotations': [{'iscrowd': 0, 'bbox': [3121, 44, 80, 80], 'category_id': 2, 'bbox_mode': <BoxMode.XYWH_ABS: 1>}, {'iscrowd': 0, 'bbox': [1373, 689, 80, 80], 'category_id': 2, 'bbox_mode': <BoxMode.XYWH_ABS: 1>}, {'iscrowd': 0, 'bbox': [2228, 493, 80, 80], 'category_id': 1, 'bbox_mode': <BoxMode.XYWH_ABS: 1>}, {'iscrowd': 0, 'bbox': [1603, 652, 80, 80], 'category_id': 1, 'bbox_mode': <BoxMode.XYWH_ABS: 1>}, {'iscrowd': 0, 'bbox': [1684, 639, 80, 80], 'category_id': 1, 'bbox_mode': <BoxMode.XYWH_ABS: 1>}, {'iscrowd': 0, 'bbox': [1757, 720, 80, 80], 'category_id': 1, 'bbox_mode': <BoxMode.XYWH_ABS: 1>}, {'iscrowd': 0, 'bbox': [1069, 495, 80, 80], 'category_id': 1, 'bbox_mode': <BoxMode.XYWH_ABS: 1>}]}, {'file_name': 'val/ssd_20190101_06.png', 'height': 1201, 'width': 3600, 'image_id': 1, 'annotations': [{'iscrowd': 0, 'bbox': [3132, 27, 80, 80], 'category_id': 2, 'bbox_mode': <BoxMode.XYWH_ABS: 1>}, {

In [ ]:
!ls

'=2.0.1'	    fval80.json        modnew.json     R50-FPN-3x-bb80
 b80-R101-DC5-3x    labtrain.json      modtrain.json   R50-FPN-3x-bb80-run2
 bb80-FPN-1x	    labval.json        modval.json     train
 flip-C4-R101-b80   Mod-C4-R101-b80    output	       val
 ftrain80.json	    Mod-DC5-R101-b80   R50-DC5-1x


In [ ]:
from detectron2.modeling import build_model
model = build_model(cfg)

In [ ]:
from detectron2.checkpoint import DetectionCheckpointer
DetectionCheckpointer(model).load('Mod-DC5-R101-b80/model_final.pth')

{'iteration': 1999,
 'optimizer': {'param_groups': [{'dampening': 0,
    'initial_lr': 0.002,
    'lr': 0.002,
    'momentum': 0.9,
    'nesterov': False,
    'params': [0],
    'weight_decay': 0.0001},
   {'dampening': 0,
    'initial_lr': 0.002,
    'lr': 0.002,
    'momentum': 0.9,
    'nesterov': False,
    'params': [1],
    'weight_decay': 0.0001},
   {'dampening': 0,
    'initial_lr': 0.002,
    'lr': 0.002,
    'momentum': 0.9,
    'nesterov': False,
    'params': [2],
    'weight_decay': 0.0001},
   {'dampening': 0,
    'initial_lr': 0.002,
    'lr': 0.002,
    'momentum': 0.9,
    'nesterov': False,
    'params': [3],
    'weight_decay': 0.0001},
   {'dampening': 0,
    'initial_lr': 0.002,
    'lr': 0.002,
    'momentum': 0.9,
    'nesterov': False,
    'params': [4],
    'weight_decay': 0.0001},
   {'dampening': 0,
    'initial_lr': 0.002,
    'lr': 0.002,
    'momentum': 0.9,
    'nesterov': False,
    'params': [5],
    'weight_decay': 0.0001},
   {'dampening': 0,
    'in

In [ ]:
!ls

'=2.0.1'   drive   sample_data


In [ ]:
import torch
from PIL import Image


image = Image.open('test/ssd_20201006_00.png')
X = torch.from_numpy(np.array(image))

print(type(X))
exit()

# inputs = [{image: X}]

# outputs = model(inputs)

FileNotFoundError: ignored